# สร้าง cappi แต่ละสถานีแบบเลือก 1 ชั่วโมงแบบ mean dbz แบบ linear _run_batch
* ขจัด noise กับ clutter
* ขจัด artifact flares และ interpolate เติมช่องว่างกลับ
* ปรับแก้การลดทอนสัญญาณ
* รวมค่าการสะท้อนราย 15 นาที ให้รายชั่วโมง ด้วยการเฉลี่ยใช้ค่า raw z
* แปลงค่า raw z เป็น dBZ
* แปลงเวลาเป็น Local time

In [1]:
'''
2024.09.22
โค้ดนี้พัฒนาโดย รองศาสตราจารย์ ดร. นัฐพล มหาวิค ภาควิชาทรัพยากรธรรมชาติและสิ่งแวดล้อม คณะเกษตรศาสตร์ฯ มหาวิทยาลัยนเรศวร 
ในงานวิจัย เรื่อง "การวิจัยและพัฒนาผลิตภัณฑ์โมเสคฝนประมาณค่าจากเรดาร์ตรวจอากาศในพื้นที่ระดับลุ่มน้ำของประเทศไทยด้วยเทคโนโลยีภูมิสารสนเทศรหัสเปิด"
สนับสนุนทุนวิจัยโดยสํานักงานการวิจัยแห่งชาติ (วช.)  แผนงานการวิจัยและนวัตกรรมแผนงานด้านการบริหารจัดการภัยพิบัติทางธรรมชาติ 
ประจำปีงบประมาณ 2566  ตามสัญญา เลขที่ N25A660467 ผู้นำโค้ดนี้ไปใช้หรือดัดแปลงควรอ้างอิงงานวิจัยชิ้นนี้ตามหลักเกณฑ์การอ้างอิงสากล
เรียนหลักการเรดาร์และภูมิสารสนเทศ ที่ https://www.youtube.com/@Nattapon_Mahavik/playlists
หนังสือเรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/144567
หนังสือออนไลน์เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/205129
ติดต่อ nattaponm@nu.ac.th
'''

# สำเร็จ แบบขจัด artifact flares และ interpolate เติมช่องว่างกลับ
import os
import numpy as np
import pyart
import xarray as xr
import rioxarray
from datetime import datetime
import copy
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.patches import Circle
from matplotlib.gridspec import GridSpec
import math
import rasterio
from pytz import timezone, utc
import warnings
import shutil
from scipy.ndimage import label


# Suppress the DeprecationWarning for Barnes weighting function
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pyart.map.gates_to_grid")

def utc2local_hr(time_rad):
    from_zone = utc
    to_zone = timezone('Asia/Bangkok')  # Adjust this to your local timezone
    utc_time = datetime.strptime(time_rad, "%Y%m%d%H")
    utc_time = utc_time.replace(tzinfo=from_zone)
    local_time = utc_time.astimezone(to_zone)
    return local_time.strftime("%Y%m%d%H")

def apply_snr_filter(radar, refl_field='reflectivity', toa=2500.0, snr_min=1, snr_max=100):
    snr = pyart.retrieve.calculate_snr_from_reflectivity(radar, refl_field=refl_field, toa=toa)
    radar.add_field('signal_to_noise_ratio', snr, replace_existing=True)
    
    gtfilter = pyart.filters.moment_and_texture_based_gate_filter(radar, phi_field='differential_phase')
    gtfilter.exclude_below('signal_to_noise_ratio', snr_min)
    gtfilter.exclude_above('signal_to_noise_ratio', snr_max)
    
    radar.add_field_like(refl_field, 'filtered_reflectivity', radar.fields[refl_field]['data'].copy())
    radar.fields['filtered_reflectivity']['data'] = np.ma.masked_where(gtfilter.gate_excluded, radar.fields['filtered_reflectivity']['data'])
    
    return radar

def detect_and_remove_flares_with_protection(radar, refl_field='filtered_reflectivity', reflectivity_threshold=10, length_threshold_km=150, protect_threshold=5):
    reflectivity = radar.fields[refl_field]['data']
    range_dist = radar.range['data']
    
    flare_mask = np.zeros(reflectivity.shape, dtype=bool)
    removed_coords = []
    
    for az_idx in range(reflectivity.shape[0]):
        radial_slice = reflectivity[az_idx, :]
        high_reflectivity = radial_slice > reflectivity_threshold
        labeled_array, num_features = label(high_reflectivity)
        
        for label_num in range(1, num_features + 1):
            segment = labeled_array == label_num
            segment_start_idx = np.where(segment)[0][0]
            segment_end_idx = np.where(segment)[0][-1]
            length_km = (range_dist[segment_end_idx] - range_dist[segment_start_idx]) / 1000.0
            
            if np.any(radial_slice[segment] <= protect_threshold):
                continue
            
            if length_km >= length_threshold_km:
                flare_mask[az_idx, :] = flare_mask[az_idx, :] | segment
                removed_coords.extend([(az_idx, idx) for idx in np.where(segment)[0]])
    
    masked_reflectivity = np.ma.masked_where(flare_mask, reflectivity)
    radar.add_field_like(refl_field, 'flare_removed_reflectivity', masked_reflectivity)
    
    return removed_coords, radar

def interpolate_missing_data(radar, removed_coords, refl_field='flare_removed_reflectivity'):
    reflectivity = radar.fields[refl_field]['data']
    
    for az_idx, range_idx in removed_coords:
        nearby_valid_values = []
        
        for az_offset in [-1, 0, 1]:
            nearby_az_idx = az_idx + az_offset
            if 0 <= nearby_az_idx < reflectivity.shape[0]:
                if not reflectivity.mask[nearby_az_idx, range_idx]:
                    nearby_valid_values.append(reflectivity[nearby_az_idx, range_idx])
        
        if nearby_valid_values:
            reflectivity[az_idx, range_idx] = np.mean(nearby_valid_values)
    
    radar.fields[refl_field]['data'] = reflectivity
    return radar

# Modify the existing snr_atten_cor function
def snr_atten_cor(radar):
    try:
        if 'reflectivity' not in radar.fields:
            print("Error: 'reflectivity' field not found in radar data.")
            return None

        # Apply SNR filter
        radar = apply_snr_filter(radar, refl_field='reflectivity', toa=2500.0, snr_min=1, snr_max=100)
        
        # Detect and remove flares
        try:
            removed_coords, radar = detect_and_remove_flares_with_protection(
                radar, 
                refl_field='filtered_reflectivity', 
                reflectivity_threshold=10, 
                length_threshold_km=150, 
                protect_threshold=5
            )
        
            # Interpolate missing data
            radar = interpolate_missing_data(radar, removed_coords, refl_field='flare_removed_reflectivity')
        except Exception as e:
            print(f"Error in flare detection or interpolation: {str(e)}")
            print("Proceeding with 'filtered_reflectivity'")
            radar.add_field('flare_removed_reflectivity', radar.fields['filtered_reflectivity'], replace_existing=True)

        if 'differential_phase' in radar.fields:
            phi_field = 'differential_phase'
        elif 'uncorrected_differential_phase' in radar.fields:
            phi_field = 'uncorrected_differential_phase'
        else:
            phi_field = None
            print("Warning: Neither 'differential_phase' nor 'uncorrected_differential_phase' found. Proceeding without phase correction.")
        
        ncp_values = np.ones((radar.nrays, radar.ngates))
        ncp = pyart.config.get_metadata('normalized_coherent_power')
        ncp['data'] = ncp_values
        radar.add_field('normalized_coherent_power', ncp)
        
        if phi_field:
            phidp, kdp = pyart.correct.phase_proc_lp(radar, 0.0, LP_solver='pyglpk', debug=True, phidp_field=phi_field)
            radar.add_field('proc_dp_phase_shift', phidp)
            radar.add_field('recalculated_diff_phase', kdp)
            
            spec_at, cor_z = pyart.correct.calculate_attenuation(
                radar,
                0,
                fzl=4500.0,
                refl_field="flare_removed_reflectivity",
                ncp_field="normalized_coherent_power",
                rhv_field="cross_correlation_ratio",
                phidp_field="proc_dp_phase_shift"
            )
            radar.add_field("specific_attenuation", spec_at)
            radar.add_field("corrected_filtered_reflectivity", cor_z)
        else:
            print("No phase field found. Using 'flare_removed_reflectivity' as final product.")
            radar.add_field('corrected_filtered_reflectivity', radar.fields['flare_removed_reflectivity'], replace_existing=True)
        
        return radar
    except Exception as e:
        print(f"Error in snr_atten_cor: {str(e)}")
        if 'flare_removed_reflectivity' in radar.fields:
            print("Using 'flare_removed_reflectivity' as 'corrected_filtered_reflectivity'.")
            radar.add_field('corrected_filtered_reflectivity', radar.fields['flare_removed_reflectivity'], replace_existing=True)
            return radar
        elif 'filtered_reflectivity' in radar.fields:
            print("Using 'filtered_reflectivity' as 'corrected_filtered_reflectivity'.")
            radar.add_field('corrected_filtered_reflectivity', radar.fields['filtered_reflectivity'], replace_existing=True)
            return radar
        elif 'reflectivity' in radar.fields:
            print("Using 'reflectivity' as 'corrected_filtered_reflectivity'.")
            radar.add_field('corrected_filtered_reflectivity', radar.fields['reflectivity'], replace_existing=True)
            return radar
        else:
            print("No suitable reflectivity field found. Skipping this radar data.")
            return None

def gridding_cappi_convert_grid2xarray(radar, height_km=2):
    lat_0 = radar.latitude['data'][0]
    lon_0 = radar.longitude['data'][0]
    
    shape = (1, 241, 241)
    height_m = height_km * 1000
    
    '''
    grid = pyart.map.grid_from_radars(
        radar,
        grid_shape=shape,
        grid_limits=((height_m, height_m), (-240000, 240000), (-240000, 240000)),
        grid_origin=(lat_0, lon_0),
        fields=['corrected_filtered_reflectivity'],
        weighting_function='Barnes2',
        min_radius=1000,
        max_radius=240000,
        roi_func='constant',
        constant_roi=5000
    )
    '''
    
 
    grid = pyart.map.grid_from_radars(
        radar,
        grid_shape=shape,
        grid_limits=((height_m, height_m), (-240000, 240000), (-240000, 240000)),
        grid_origin=(lat_0, lon_0),
        fields=['corrected_filtered_reflectivity']
    )
   
    
    ds = grid.to_xarray()
    grid_data = ds['corrected_filtered_reflectivity'][0, 0, :, :]
    return grid_data

def save_geotif(cappi_max_hr, path_outfile):
    cappi_max_hr = cappi_max_hr.rename('reflectivity')
    cappi_max_hr = cappi_max_hr.to_dataset().squeeze().set_index(x="lon", y="lat")
    cappi_max_hr.rio.set_spatial_dims(x_dim='x', y_dim='y', inplace=True)
    cappi_max_hr.rio.set_crs("EPSG:4326")
    cappi_max_hr.rio.write_crs("EPSG:4326", inplace=True)
    cappi_max_hr.rio.write_transform()
    cappi_max_hr.rio.write_coordinate_system()
    cappi_max_hr.rio.to_raster(path_outfile)
    cappi_max_hr.close()

def dbz_to_linear(dbz):
    """Convert dBZ to linear scale, handling NaN values."""
    return np.where(np.isfinite(dbz), 10 ** (dbz / 10), np.nan)

def linear_to_dbz(linear):
    """Convert linear scale to dBZ, handling NaN and zero values."""
    return np.where(linear > 0, 10 * np.log10(linear), np.nan)

def process_single_radar(radar_name, base_dir, time_hr):
    print(f"Starting to process {radar_name}")
    dir_path = os.path.join(base_dir, radar_name)
    
    if not os.path.exists(dir_path):
        print(f"Error: Directory does not exist: {dir_path}")
        return None, {'radar': radar_name, 'hour': time_hr, 'file_count': 0, 'complete': False, 'files_present': [], 'files_missing': [], 'files_processed': 0, 'differential_phase_present': False}
    
    fn_all = [file for file in os.listdir(dir_path) if file.startswith(f"{radar_name}240@{time_hr}") and file.endswith('.uf.bz2')]
    
    print(f"Files found for {radar_name}: {fn_all}")
    
    report = {
        'radar': radar_name,
        'hour': time_hr,
        'file_count': len(fn_all),
        'complete': len(fn_all) == 4,
        'files_present': fn_all,
        'files_missing': [],
        'files_processed': 0,
        'differential_phase_present': False
    }
    
    if len(fn_all) != 4:
        expected_files = [f"{radar_name}240@{time_hr}{m:02d}.uf.bz2" for m in range(0, 60, 15)]
        report['files_missing'] = list(set(expected_files) - set(fn_all))
    
    radar_list = []
    for file_15min in fn_all:
        file_path = os.path.join(dir_path, file_15min)
        temp_file = None
        try:
            if radar_name == 'NRT' and file_path.endswith('.bz2'):
                temp_file = file_path[:-4]  # Remove .bz2 extension
                shutil.copy2(file_path, temp_file)
                file_path = temp_file
            
            radar = pyart.io.read(file_path)
            radar.metadata['file_name'] = file_15min
            processed_radar = snr_atten_cor(radar)
            if processed_radar is not None:
                radar_list.append(processed_radar)
                report['files_processed'] += 1
                if 'differential_phase' in processed_radar.fields or 'uncorrected_differential_phase' in processed_radar.fields:
                    report['differential_phase_present'] = True
            print(f"Successfully processed {file_15min}")
        except Exception as e:
            print(f"Error processing {file_15min}: {str(e)}")
        finally:
            if temp_file and os.path.exists(temp_file):
                os.remove(temp_file)

    if not radar_list:
        print(f"No valid radar data found for {radar_name} at {time_hr}")
        return None, report
    
    try:
        print(f'Creating 2 km CAPPI using {len(radar_list)} files...')
        cappi_list = []
        for radar in radar_list:
            radar_copy = copy.deepcopy(radar)
            grid = gridding_cappi_convert_grid2xarray(radar_copy)
            cappi_list.append(grid)
        
        print('Calculating mean dBZ...')
        linear_cappis = [dbz_to_linear(cappi.values) for cappi in cappi_list]
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=RuntimeWarning, message="Mean of empty slice")
            linear_mean = np.nanmean(linear_cappis, axis=0)
        
        cappi_mean = xr.DataArray(
            linear_to_dbz(linear_mean),
            coords=cappi_list[0].coords,
            dims=cappi_list[0].dims,
            attrs=cappi_list[0].attrs,
            name='reflectivity'
        )
        
        print(f'Saving results for {radar_name}...')
        local_time_hr = utc2local_hr(time_hr)
        output_dir = f'../3output/0mosaic/0ztest/0processing_single_cappi/00outp_cappi_2km_{radar_name}'
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, f'{local_time_hr}.tif')
        save_geotif(cappi_mean, output_file)
        
        if os.path.exists(output_file):
            print(f"Successfully saved GeoTIFF for {radar_name}: {output_file}")
            return output_file, report
        else:
            print(f"Failed to save GeoTIFF for {radar_name}")
            return None, report
    except Exception as e:
        print(f"Error in CAPPI creation or saving for {radar_name}: {str(e)}")
        return None, report
    
def visualize_results(results, time_hr, basin_shape_path, radars, utc2local_hr):
    print(f"Radars in results: {list(results.keys())}")
    print(f"Radars in radars dictionary: {list(radars.keys())}")
    
    missing_radars = set(radars.keys()) - set(results.keys())
    if missing_radars:
        print(f"Missing radars: {missing_radars}")

    basin_shape = gpd.read_file(basin_shape_path)
    
    n_radars = len(results)
    n_cols = min(3, n_radars)  # Max 3 columns
    n_rows = math.ceil(n_radars / n_cols)
    
    fig = plt.figure(figsize=(8 * n_cols, 7 * n_rows))
    gs = GridSpec(n_rows, n_cols, figure=fig, wspace=0.15, hspace=0.20)
    
    for i, (radar_name, file_path) in enumerate(results.items()):
        print(f"Visualizing {radar_name} from file: {file_path}")
        try:
            with rasterio.open(file_path) as src:
                dbz = src.read(1)
                cols, rows = np.meshgrid(np.arange(src.width), np.arange(src.height))
                lon, lat = rasterio.transform.xy(src.transform, rows, cols)
            
            ax = fig.add_subplot(gs[i // n_cols, i % n_cols])
            
            im = ax.pcolormesh(lon, lat, dbz, cmap='jet', shading='auto', vmin=0, vmax=50)
            
            basin_shape.plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=1)
            
            ax.set_title(f"Mean dBZ for {radar_name} Radar", fontsize=16)
            ax.set_xlim(np.min(lon), np.max(lon))
            ax.set_ylim(np.min(lat), np.max(lat))
            
            ax.set_xlabel("Longitude", fontsize=12)
            ax.set_ylabel("Latitude", fontsize=12)
            
            ax.tick_params(axis='both', which='major', labelsize=10)
            
            radar_coords = radars[radar_name]["coords"]
            ax.plot(radar_coords[0], radar_coords[1], 'ko', markersize=5, label='Radar')
            
            for radius in [120, 240]:
                circle = Circle((radar_coords[0], radar_coords[1]), radius/111, 
                                fill=False, color='black', linestyle='--', linewidth=1)
                ax.add_artist(circle)
            
            ax.legend(fontsize=10)
            print(f"Successfully visualized {radar_name}")
        except Exception as e:
            print(f"Error visualizing {radar_name}: {str(e)}")
    
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    cbar = fig.colorbar(im, cax=cbar_ax, extend='max')
    cbar.set_label("Mean dBZ", fontsize=16)
    cbar.ax.tick_params(labelsize=12)
    
    local_time_hr = utc2local_hr(time_hr)
    plt.suptitle(f"Mean dBZ for Thai Radars ({local_time_hr} local time)", fontsize=22, y=0.98)
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    plt.savefig(f'../3output/0mosaic/0ztest/0processing_single_cappi/cappi_2km_{local_time_hr}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
def write_report(reports, output_file):
    with open(output_file, 'w') as f:
        f.write("Radar,Hour (UTC),Hour (Local),File Count,Complete,Files Present,Files Missing,Files Processed,Differential Phase Present,Total Radars Processed\n")
        for report in reports:
            local_time = utc2local_hr(report['hour'])
            f.write(f"{report['radar']},{report['hour']},{local_time},{report['file_count']},{report['complete']},")
            f.write(f"\"{','.join(report['files_present'])}\",\"{','.join(report['files_missing'])}\",")
            f.write(f"{report['files_processed']},{report['differential_phase_present']},{report['total_radars_processed']}\n")

def main():
    base_dir = '../1data/0radar/3Mosaic_hourly_1hour_sontihn/'
    #basin_shape_path = '../1data/1GIS/0base_map_gis/Chaophraya_Basins_others_Boundary_WGS84Geo.shp'
    basin_shape_path = '../1data/1GIS/0base_map_gis/MainBasin_ONWR_Law_WGS84Geo.shp'
    
    radars = {
        "CHN": {'coords': (100.191263, 15.157852, 40.0), 'el': 0.5},
        "CMP": {'coords': (99.188203, 10.493099, 32.0), 'el': 0.0},
        "CRI": {'coords': (99.881593, 19.961471, 444.0), 'el': 1.08},
        "KKN": {'coords': (102.785881, 16.4625, 217.0), 'el': 0.42},
        "KRB": {'coords': (98.97806, 8.101389, 52.0), 'el': 0.8},
        "LMP": {'coords': (99.038728, 18.566602, 328.0), 'el': 1.67},
        "NRT": {'coords': (101.825165, 6.426888, 33.0), 'el': 0.5},
        "NAN": {'coords': (100.788902, 19.124102, 260.0), 'el': 1.41},
        "PHS": {'coords': (100.217964, 16.775408, 72.0), 'el': 0.5},
        "PKT": {'coords': (98.329462, 8.133407, 281.0), 'el': 0.02},
        "SNK": {'coords': (104.132591, 17.156363, 196.0), 'el': 0.42},
        "STP": {'coords': (100.459996, 7.449996, 33.0), 'el': 0.5},
        "SVP": {'coords': (100.766541, 13.684727, 39.0), 'el': 1.5},
        "URT": {'coords': (104.870004, 15.245, 157.0), 'el': 0.41},
    } 
    
    times_file_path = './0variables_cappi/cappi_hourly_times.txt'
    with open(times_file_path, 'r') as file:
        time_hours = [line.strip() for line in file if line.strip()]
    
    for time_hr in time_hours:
        print(f"\nProcessing time: {time_hr} UTC")
        results = {}
        reports = []
        total_processed = 0
        
        for radar_name in radars.keys():
            print(f"\nProcessing radar: {radar_name}")
            result, report = process_single_radar(radar_name, base_dir, time_hr)
            reports.append(report)
            
            if result is not None:
                results[radar_name] = result
                total_processed += 1
                print(f"Added {radar_name} to results dictionary")
            else:
                print(f"No valid result for {radar_name}, not added to results dictionary")
        
        print(f"\nTotal radars processed: {total_processed}")
        print(f"Radars in results dictionary: {list(results.keys())}")
        print(f"Radars not in results dictionary: {set(radars.keys()) - set(results.keys())}")
        
        # Update reports with the correct total processed count
        for report in reports:
            report['total_radars_processed'] = total_processed
        
        if results:
            visualize_results(results, time_hr, basin_shape_path, radars, utc2local_hr)
        else:
            print("No results to visualize")
        
        local_time_hr = utc2local_hr(time_hr)
        report_file_path = f'../3output/0mosaic/0ztest/0report_each_radars/radar_data_report_{local_time_hr}.csv'
        write_report(reports, report_file_path)
        
        print(f"Processing completed for time {time_hr} UTC ({local_time_hr} local time)")

if __name__ == "__main__":
    main()
 


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


Processing time: 2018071500 UTC

Processing radar: CHN
Starting to process CHN
Files found for CHN: ['CHN240@201807150000.uf.bz2', 'CHN240@201807150015.uf.bz2', 'CHN240@201807150030.uf.bz2', 'CHN240@201807150045.uf.bz2']
Unfolding


C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\pyart\correct\phase_proc.py:170: RuntimeWarning: invalid value encountered in sqrt
  _noise = smooth_and_trim(np.sqrt((line - signal) ** 2), window_len=wl)
C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\pyart\correct\phase_proc.py:171: RuntimeWarning: divide by zero encountered in divide
  return abs(signal) / _noise


Exec time:  2.0683369636535645
Doing  0
Doing  1
Doing  2
Doing  3


C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Successfully processed CHN240@201807150000.uf.bz2
Unfolding
Exec time:  2.0446529388427734
Doing  0
Doing  1
Doing  2
Doing  3
Successfully processed CHN240@201807150015.uf.bz2
Unfolding
Exec time:  2.0943658351898193
Doing  0
Doing  1
Doing  2
Doing  3
Successfully processed CHN240@201807150030.uf.bz2
Unfolding
Exec time:  2.1323390007019043
Doing  0
Doing  1
Doing  2
Doing  3
Successfully processed CHN240@201807150045.uf.bz2
Creating 2 km CAPPI using 4 files...
Calculating mean dBZ...
Saving results for CHN...
Successfully saved GeoTIFF for CHN: ../3output/0mosaic/0ztest/0processing_single_cappi/00outp_cappi_2km_CHN\2018071507.tif
Added CHN to results dictionary

Processing radar: CMP
Starting to process CMP
Files found for CMP: ['CMP240@201807150000.uf.bz2', 'CMP240@201807150015.uf.bz2', 'CMP240@201807150030.uf.bz2', 'CMP240@201807150045.uf.bz2']
No phase field found. Using 'flare_removed_reflectivity' as final product.
Successfully processed CMP240@201807150000.uf.bz2
No phase fiel

...

Successfully processed SNK240@201807150000.uf.bz2
Unfolding
Exec time:  1.677624225616455
Doing  0
Doing  1
Doing  2
Doing  3


C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\pyart\correct\attenuation.py:952: RuntimeWarning: overflow encountered in multiply
  reflectivity_linear * self_cons_number /
C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\pyart\correct\attenuation.py:953: RuntimeWarning: overflow encountered in multiply
  (I_indef[0] + self_cons_number * I_indef))
C:\Users\Admin\anaconda3\envs\mossf\lib\site-packages\pyart\correct\attenuation.py:946: RuntimeWarning: overflow encountered in double_scalars
  self_cons_number = 10.0 ** (0.1 * beta * a_coef * phidp_max) - 1.0


Successfully processed SNK240@201807150015.uf.bz2
Unfolding
Exec time:  1.7593276500701904
Doing  0
Doing  1
Doing  2
Doing  3
Successfully processed SNK240@201807150030.uf.bz2
Unfolding
Exec time:  1.6815009117126465
Doing  0
Doing  1
Doing  2
Doing  3
Successfully processed SNK240@201807150045.uf.bz2
Creating 2 km CAPPI using 4 files...
Calculating mean dBZ...
Saving results for SNK...
Successfully saved GeoTIFF for SNK: ../3output/0mosaic/0ztest/0processing_single_cappi/00outp_cappi_2km_SNK\2018071507.tif
Added SNK to results dictionary

Processing radar: STP
Starting to process STP
Files found for STP: ['STP240@201807150000.uf.bz2', 'STP240@201807150015.uf.bz2', 'STP240@201807150030.uf.bz2', 'STP240@201807150045.uf.bz2']
No phase field found. Using 'flare_removed_reflectivity' as final product.
Successfully processed STP240@201807150000.uf.bz2
No phase field found. Using 'flare_removed_reflectivity' as final product.
Successfully processed STP240@201807150015.uf.bz2
No phase field 

C:\Users\Admin\AppData\Local\Temp\ipykernel_22340\2409831354.py:383: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Processing completed for time 2018071500 UTC (2018071507 local time)
